In [1]:
!ls /home/askyourcorpus/aditya_qc_experiments/QueryCraft-fork/output/evalResultsDB2

codellama_codellama-34b.csv	    ibm_granite-8b-code.csv
deepseek-ai_deepseek-coder-33b.csv  kaist-ai_prometheus-8x7b-v2.csv
ibm_granite-13b-instruct-v2.csv     meta-llama-3-70b.csv
ibm_granite-20b-code.csv	    mixtral-8x7b-instruct-v01.csv
ibm_granite-34b-code.csv


In [2]:
import pandas as pd
import re
import warnings
from sql_metadata import Parser
from matplotlib import pyplot as plt
import json
import logging
import ibm_db

import sqlite3
from tqdm import tqdm
from itertools import product
from collections import defaultdict
import random
from datetime import datetime
import os
from math import ceil
import random
import sys
from datasets import load_dataset
import os



warnings.filterwarnings('ignore')

In [3]:
query_keywords_list = [
    "TOP","EXISTS","INTERSECT","SELECT","DISTINCT","RANK","AS",
    "WHERE","AND","OR","BETWEEN","LIKE","COUNT","SUM","AVG",
    "MIN","MAX","GROUP BY","ORDER BY","DESC","OFFSET","FETCH",
    "INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION",
    "HAVING","JOIN"
]
aggregate_keywords = ["COUNT","SUM","AVG","MIN","MAX","TOP"]
rank_keywords = ["RANK"]
fillter_keywords = ["GROUP BY","ORDER BY","FILTER","HAVING","EXISTS"]
join_keywords = [
    "JOIN","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION","INTERSECT"
]
orderby_keywords = ["ORDER BY"]
groupby_keywords = ["GROUP BY"]
where_keywords =["WHERE"]
date_keywords =[
    "NOW","GETDATE","CURRENT_TIMESTAMP","DATEDIFF","DATEADD","YEAR","DAY","MONTH"
]
keyword_pattern = re.compile(r'\b(?:' + '|'.join(query_keywords_list) + r')\b', re.IGNORECASE)

def calculate_classification_new(df):
    ## Create new 3 columns 
    df["Expected count"] = df.index
    df["Expected difficulty"] = df.index
    df["Expected classification_new"] = df.index
    for index, row in df.iterrows():
        sql = row["expected query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in orderby_keywords:
                    classification = "ORDER BY"
                elif keyword in groupby_keywords:
                    classification = "GROUP BY"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE/RATIO"
                elif keyword in join_keywords:
                    classification = "JOIN"
                elif keyword in where_keywords:
                    classification = "WHERE"
                elif keyword in date_keywords:
                    classification = "DATE"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Expected difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Expected difficulty'] ="moderate"
        else:
            df.at[index,'Expected difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Expected classification_new'] = classification
        df.at[index,'Expected count'] =count


    ## Create new 3 columns 
    df["Predicted count"] = df.index
    df["Predicted difficulty"] = df.index
    df["Predicted classification_new"] = df.index
    for index, row in df.iterrows():
        sql = row["generated query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in orderby_keywords:
                    classification = "ORDER BY"
                elif keyword in groupby_keywords:
                    classification = "GROUP BY"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE/RATIO"
                elif keyword in join_keywords:
                    classification = "JOIN"
                elif keyword in where_keywords:
                    classification = "WHERE"
                elif keyword in date_keywords:
                    classification = "DATE"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Predicted difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Predicted difficulty'] ="moderate"
        else:
            df.at[index,'Predicted difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Predicted classification_new'] = classification
        df.at[index,'Predicted count'] =count
    return df

def calculate_classification(df):
    ## Create new 3 columns 
    df["Expected count"] = df.index
    df["Expected difficulty"] = df.index
    df["Expected classification"] = df.index
    for index, row in df.iterrows():
        sql = row["expected query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in rank_keywords:
                    classification = "RANK"
                elif keyword in fillter_keywords:
                    classification = "FILTER"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE"
                elif keyword in join_keywords:
                    classification = "JOIN"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Expected difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Expected difficulty'] ="moderate"
        else:
            df.at[index,'Expected difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Expected classification'] = classification
        df.at[index,'Expected count'] =count

        ## Create new 3 columns 
    df["Predicted count"] = df.index
    df["Predicted difficulty"] = df.index
    df["Predicted classification"] = df.index
    for index, row in df.iterrows():
        sql = row["generated query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in rank_keywords:
                    classification = "RANK"
                elif keyword in fillter_keywords:
                    classification = "FILTER"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE"
                elif keyword in join_keywords:
                    classification = "JOIN"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Predicted difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Predicted difficulty'] ="moderate"
        else:
            df.at[index,'Predicted difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Predicted classification'] = classification
        df.at[index,'Predicted count'] =count
    return df

def tokenize_sql_query(sql_query):
    token_list = []
    try:
        for token in Parser(sql_query).tokens:
            token_list.append(str(token.value))
    except:
        pass
    return token_list



def get_category(query):
    query_keywords_list = ["TOP","EXISTS","INTERSECT","SELECT","DISTINCT","TOP","RANK","AS","WHERE","AND","OR","BETWEEN","LIKE","COUNT","SUM","AVG","MIN","MAX","GROUP BY","ORDER BY","DESC","OFFSET","FETCH","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION","HAVING","JOIN"]
    aggregate_keywords = ["COUNT","SUM","AVG","MIN","MAX","TOP"]
    rank_keywords = ["RANK"]
    fillter_keywords = ["GROUP BY","ORDER BY","FILTER","HAVING","EXISTS"]
    join_keywords = ["JOIN","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION","INTERSECT"]

    count =0
    orderby_keywords = ["ORDER BY"]
    groupby_keywords = ["GROUP BY"]
    where_keywords =["WHERE"]
    date_keywords =["NOW","GETDATE","CURRENT_TIMESTAMP","DATEDIFF","DATEADD","YEAR","DAY","MONTH"]
    for keyword in query_keywords_list:
        if isinstance(query,str) and keyword in query:
            count=count+1
            if keyword in orderby_keywords:
                classification = "ORDER BY"
            elif keyword in groupby_keywords:
                classification = "GROUP BY"
            elif keyword in aggregate_keywords:
                classification = "AGGREGATE/RATIO"
            elif keyword in join_keywords:
                classification = "JOIN"
            elif keyword in where_keywords:
                classification = "WHERE"
            elif keyword in date_keywords:
                classification = "DATE"
    if count < 6:
        classification="simple"
    elif count > 5 and count < 9:
        classification="moderate"
    else:
        classification ="challenging"
    if classification == '':
        classification = 'SELECT'
    return classification


In [4]:
base_path = "../input/sirion_dataset_v3/"

model_paths = os.listdir(base_path)


dict_accuracy_overall = {}
dict_latency_overall = {}


for model_path in model_paths:
    
    df = pd.read_csv(base_path+model_path)
    model_id = model_path.split(".csv")[0]

    
    df["predicted_query_toks"] = df["generated query"].apply(tokenize_sql_query).astype(str).replace(","," ")
    df["expected_query_toks"] = df["expected query"].apply(tokenize_sql_query).astype(str).replace(","," ")
    
    df['expected_sql_classification'] = df['expected query'].apply(lambda sql: get_category(sql))
    df['generated_sql_classification'] = df['generated query'].apply(lambda sql: get_category(sql))
    df = calculate_classification(df,)
    df = calculate_classification_new(df)
    df.to_csv("../output/inference_sirion_v3/{}.csv".format(model_id),index=False)

    filtered_df = df
    grouped = df.groupby('generated_sql_classification')
    execution_accuracy = {}
    
    cat_df = pd.DataFrame(columns=["category","Total evaluation records for category","Total correct responses","Execution accuracy %"])
    for name, group in grouped:
        cat_list=[]
        total_true = group['evalScore'].sum()
        total_records = len(group)
        print(total_true,total_records,"##")
        accuracy = total_true / total_records
        execution_accuracy[name] = accuracy
        cat_list.append(name)
        cat_list.append(total_records)
        cat_list.append(total_true)
        cat_list.append(accuracy)
        print(cat_list)
        new_row_df = pd.DataFrame([cat_list], columns=cat_df.columns)
        cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)
    
    
    tot_list = []
    category = "TOTAL"
    total_eval_records = filtered_df.shape[0]
    evalScore_counts = df[df['evalScore'] == True]['evalScore'].value_counts()[0]
    exec_accuracy =  evalScore_counts/total_eval_records
    tot_list.append(category)
    tot_list.append(total_eval_records)
    tot_list.append(evalScore_counts)
    tot_list.append(exec_accuracy)
    new_row_df = pd.DataFrame([tot_list], columns=cat_df.columns)
    cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)
    cat_df['Execution accuracy %'] =round(cat_df['Execution accuracy %']*100,2)
    
    cat_df.to_csv("../output/inference_sirion_v3/accumulated_accuracy/{}.csv".format(model_id),index=False)
    dict_accuracy_overall[model_id] =  cat_df[cat_df["category"]=="TOTAL"]["Execution accuracy %"].values[0]
    dict_latency_overall[model_id] = round(df['latency'].mean(),3)

Not supported query type: "SELECT payment_terms
FROM contract
WHERE id = 'CO100123'
AND document_type = 'invoice' AND keywords LIKE '%payment terms%' AND keywords LIKE '%invoice%';"
Not supported query type: "SELECT * 
FROM contract
WHERE status = 'archived' AND document_type = 'NDA' AND created_by= 'John Smith';
Not supported query type: "SELECT *
FROM contract
WHERE term_type = 'Fixed' AND created_on >= '2023-01-01';"
Not supported query type: "SELECT *
FROM contract
WHERE expiration_date >= '2024-01-01'
  AND expiration_date <= '2024-12-31';
Not supported query type: -
Not supported query type: -
Not supported query type: -
Not supported query type: "SELECT payment_terms
FROM contract
WHERE id = 'CO100123'
AND document_type = 'invoice' AND keywords LIKE '%payment terms%' AND keywords LIKE '%invoice%';"
Not supported query type: "SELECT * 
FROM contract
WHERE status = 'archived' AND document_type = 'NDA' AND created_by= 'John Smith';
Not supported query type: "SELECT *
FROM contract


0 2 ##
['challenging', 2, 0, 0.0]
1 11 ##
['moderate', 11, 1, 0.09090909090909091]
71 184 ##
['simple', 184, 71, 0.3858695652173913]


Not supported query type: -
Not supported query type: -
Not supported query type: -
Not supported query type: "SELECT payment_terms
FROM contract
WHERE id = 'CO100123'
AND document_type = 'invoice' AND keywords LIKE '%payment terms%' AND keywords LIKE '%invoice%';"
Not supported query type: "SELECT * 
FROM contract
WHERE status = 'archived' AND document_type = 'NDA' AND created_by= 'John Smith';
Not supported query type: "SELECT *
FROM contract
WHERE term_type = 'Fixed' AND created_on >= '2023-01-01';"
Not supported query type: "SELECT *
FROM contract
WHERE expiration_date >= '2024-01-01'
  AND expiration_date <= '2024-12-31';


1 13 ##
['moderate', 13, 1, 0.07692307692307693]
74 184 ##
['simple', 184, 74, 0.40217391304347827]


Not supported query type: -
Not supported query type: -
Not supported query type: -


...

0 2 ##
['challenging', 2, 0, 0.0]
1 11 ##
['moderate', 11, 1, 0.09090909090909091]
66 184 ##
['simple', 184, 66, 0.358695652173913]


Not supported query type: -
Not supported query type: -
Not supported query type: -
Not supported query type: "SELECT payment_terms
FROM contract
WHERE id = 'CO100123'
AND document_type = 'invoice' AND keywords LIKE '%payment terms%' AND keywords LIKE '%invoice%';"
Not supported query type: "SELECT * 
FROM contract
WHERE status = 'archived' AND document_type = 'NDA' AND created_by= 'John Smith';
Not supported query type: "SELECT *
FROM contract
WHERE term_type = 'Fixed' AND created_on >= '2023-01-01';"
Not supported query type: "SELECT *
FROM contract
WHERE expiration_date >= '2024-01-01'
  AND expiration_date <= '2024-12-31';


1 8 ##
['moderate', 8, 1, 0.125]
75 189 ##
['simple', 189, 75, 0.3968253968253968]


Not supported query type: -
Not supported query type: -
Not supported query type: -


0 1 ##
['challenging', 1, 0, 0.0]
1 9 ##
['moderate', 9, 1, 0.1111111111111111]
65 187 ##
['simple', 187, 65, 0.34759358288770054]


In [5]:
dict_accuracy_overall

{'codellama_codellama-34b': 36.55,
 'deepseek-ai_deepseek-coder-33b': 38.07,
 'ibm_granite-8b-code': 32.99,
 'ibm_granite-13b-instruct-v2': 1.52,
 'ibm_granite-20b-code': 24.37,
 'ibm_granite-34b-code': 37.56,
 'kaist-ai_prometheus-8x7b-v2': 34.01,
 'meta-llama-3-70b': 38.58,
 'mixtral-8x7b-instruct-v01': 33.5}

In [6]:
dict_latency_overall

{'codellama_codellama-34b': 19.912,
 'deepseek-ai_deepseek-coder-33b': 19.88,
 'ibm_granite-8b-code': 10.724,
 'ibm_granite-13b-instruct-v2': 24.707,
 'ibm_granite-20b-code': 24.317,
 'ibm_granite-34b-code': 19.184,
 'kaist-ai_prometheus-8x7b-v2': 11.414,
 'meta-llama-3-70b': 14.085,
 'mixtral-8x7b-instruct-v01': 11.144}